In [1]:
import pandas as pd
import numpy as np
import datetime
from isoweek import Week
PATH = 'dataset/rossmann/'

In [2]:
table_names = ['train', 'test', 'store', 'store_states', 'state_names', 'googletrend', 'weather']
train, test, store, store_states, state_names, googletrend, weather = [pd.read_csv(PATH + fname+'.csv', 
                                                                             low_memory=False) for fname in table_names]

# Paso StateHoliday a booleana
Puede ser discutible

In [3]:
train['StateHoliday'].value_counts()

0    986159
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

a = public holiday, b = Easter holiday, c = Christmas, 0 = None

In [4]:
train.StateHoliday = train.StateHoliday!='0'

In [5]:
test.StateHoliday = test.StateHoliday!='0'

# Función para hacer el join

In [6]:
def join_df(left, right, left_on, right_on=None):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", "_y"))

## Join entre weather y states_names

In [7]:
weather.columns

Index(['file', 'Date', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h',
       'Max_Gust_SpeedKm_h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees'],
      dtype='object')

In [8]:
weather['file'].value_counts()

Berlin                   990
BadenWuerttemberg        990
Bremen                   990
Sachsen                  990
SachsenAnhalt            990
Hessen                   990
Saarland                 990
Brandenburg              990
Bayern                   990
Thueringen               990
NordrheinWestfalen       990
SchleswigHolstein        990
Hamburg                  990
RheinlandPfalz           990
Niedersachsen            990
MecklenburgVorpommern    990
Name: file, dtype: int64

In [9]:
state_names

,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB
5,Hamburg,HH
6,Hessen,HE
7,MecklenburgVorpommern,MV
8,Niedersachsen,"HB,NI"
9,NordrheinWestfalen,NW


In [10]:
weather = join_df(weather, state_names, "file", "StateName")

In [11]:
weather.columns

Index(['file', 'Date', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h',
       'Max_Gust_SpeedKm_h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees', 'StateName', 'State'],
      dtype='object')

Notar como se agregaron las columnas

## Preprocess googletrend

In [12]:
googletrend.head()

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


In [13]:
# Armo columan date con el primer día de la semana
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]

In [14]:
googletrend.head()

,file,week,trend,Date
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30


In [15]:
# Armo columna de State con el nomnre del estado
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]

In [16]:
googletrend.head()

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN


In [17]:
googletrend['State'].value_counts()

BW    148
SL    148
NI    148
SH    148
NW    148
HE    148
HH    148
BE    148
ST    148
SN    148
BY    148
TH    148
RP    148
Name: State, dtype: int64

In [18]:
# Notar que un estado tiene un acrónimo diferente
state_names['State']

0        BW
1        BY
2        BE
3        BB
4        HB
5        HH
6        HE
7        MV
8     HB,NI
9        NW
10       RP
11       SL
12       SN
13       ST
14       SH
15       TH
Name: State, dtype: object

In [19]:
# Lo corrijo
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

# Transformo fecha

In [20]:
def add_datepart(df):
    df.Date = pd.to_datetime(df.Date)
    df["Year"] = df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Week"] = df.Date.dt.week
    df["Day"] = df.Date.dt.day
add_datepart(weather)
add_datepart(googletrend)
add_datepart(train)
add_datepart(test)

In [21]:
# Trends generales de alemania tienen el state en None
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

# Hago todo el resto de los joins

In [22]:
# Agrego acronimo de state al store
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [23]:
store.columns

Index(['Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State'],
      dtype='object')

In [24]:
# Mergeo train y store
joined_train = join_df(train, store, "Store")
len(joined_train[joined_train.StoreType.isnull()])

0

In [26]:
joined_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State'],
      dtype='object')

In [27]:
# Mergeo test y store
joined_test = join_df(test, store, "Store")
len(joined_test[joined_test.StoreType.isnull()])

0

In [28]:
joined_train = join_df(joined_train, googletrend, ["State","Year", "Week"])
len(joined_train[joined_train.trend.isnull()])

0

In [29]:
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined_test[joined_test.trend.isnull()])

0

In [30]:
joined_train = joined_train.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined_train[joined_train.trend_DE.isnull()])

0

In [37]:
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined_test[joined_test.trend_DE.isnull()])

0

In [38]:
joined_train = join_df(joined_train, weather, ["State","Date"])
len(joined_train[joined_train.Mean_TemperatureC.isnull()])

0

In [39]:
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined_test[joined_test.Mean_TemperatureC.isnull()])

0

# Fill missing values

In [42]:
# Porcentajes de nan para tener una idea
joined_train.CompetitionOpenSinceYear.isna().sum()/len(joined_train.CompetitionOpenSinceYear)

0.31787764363075827

In [43]:
joined_test.CompetitionOpenSinceYear.isna().sum()/len(joined_test.CompetitionOpenSinceYear)

0.3703271028037383

In [44]:
# Discutible que le ponga 1900?
joined_train.CompetitionOpenSinceYear = joined_train.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
joined_train.CompetitionOpenSinceMonth = joined_train.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
joined_train.Promo2SinceYear = joined_train.Promo2SinceYear.fillna(1900).astype(np.int32)
joined_train.Promo2SinceWeek = joined_train.Promo2SinceWeek.fillna(1).astype(np.int32)

In [45]:
# Discutible que le ponga 1900?
joined_test.CompetitionOpenSinceYear = joined_test.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
joined_test.CompetitionOpenSinceMonth = joined_test.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
joined_test.Promo2SinceYear = joined_test.Promo2SinceYear.fillna(1900).astype(np.int32)
joined_test.Promo2SinceWeek = joined_test.Promo2SinceWeek.fillna(1).astype(np.int32)

##  Armamos CompetitionOpenSince mergenado CompetitionOpenSinceYear, CompetitionOpenSinceMonth

In [47]:
# Pone dia 15 ya que no cuenta con ese dato
joined_train["CompetitionOpenSince"] = pd.to_datetime(joined_train.apply(lambda x: datetime.datetime(
    x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 15), axis=1))

In [48]:
# Pone dia 15 ya que no cuenta con ese dato
joined_test["CompetitionOpenSince"] = pd.to_datetime(joined_test.apply(lambda x: datetime.datetime(
    x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 15), axis=1))

In [49]:
joined_train.CompetitionOpenSince[:10]

0   2008-09-15
1   2007-11-15
2   2006-12-15
3   2009-09-15
4   2015-04-15
5   2013-12-15
6   2013-04-15
7   2014-10-15
8   2000-08-15
9   2009-09-15
Name: CompetitionOpenSince, dtype: datetime64[ns]

In [50]:
joined_train["CompetitionDaysOpen"] = joined_train.Date.subtract(joined_train["CompetitionOpenSince"]).dt.days
joined_train["CompetitionDaysOpen"][:10]

0    2510
1    2815
2    3150
3    2145
4     107
5     593
6     837
7     289
8    5463
9    2145
Name: CompetitionDaysOpen, dtype: int64

In [51]:
joined_test["CompetitionDaysOpen"] = joined_test.Date.subtract(joined_test["CompetitionOpenSince"]).dt.days
joined_test["CompetitionDaysOpen"][:10]

0     2558
1     3198
2      885
3      337
4     5511
5     2193
6     1402
7    42248
8    42248
9      551
Name: CompetitionDaysOpen, dtype: int64

In [53]:
# Corrige errores de la formula anterior
joined_train.loc[joined_train.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
joined_train.loc[joined_train.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [54]:
# Corrige errores de la formula anterior
joined_test.loc[joined_test.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
joined_test.loc[joined_test.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [56]:
# Lo pone en meses y limita a 2 años como máximo
joined_train["CompetitionMonthsOpen"] = joined_train["CompetitionDaysOpen"]//30
joined_train.loc[joined_train.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined_train.CompetitionMonthsOpen.value_counts()

24    465838
0     417638
6       6356
7       6350
4       6278
5       6261
8       6203
9       6138
10      6118
11      5971
12      5867
3       5850
13      5724
15      5699
14      5655
19      5649
2       5621
18      5619
16      5618
22      5558
17      5549
20      5500
1       5482
21      5398
23      5269
Name: CompetitionMonthsOpen, dtype: int64

In [57]:
# Lo pone en meses y limita a 2 años como máximo
joined_test["CompetitionMonthsOpen"] = joined_test["CompetitionDaysOpen"]//30
joined_test.loc[joined_test.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined_test.CompetitionMonthsOpen.value_counts()

24    21470
0     15491
23      346
17      292
16      287
13      260
1       257
10      247
11      246
14      197
18      169
12      164
8       161
2       161
4       152
5       151
20      149
21      148
22      146
3       141
15      126
9       125
19       84
7        60
6        58
Name: CompetitionMonthsOpen, dtype: int64

### Mismo proceso a promodates

In [58]:
joined_train["Promo2Since"] = pd.to_datetime(joined_train.apply(lambda x: Week(
    x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
joined_train["Promo2Days"] = joined_train.Date.subtract(joined_train["Promo2Since"]).dt.days
joined_train.loc[joined_train.Promo2Days<0, "Promo2Days"] = 0
joined_train.loc[joined_train.Promo2SinceYear<1990, "Promo2Days"] = 0
joined_train["Promo2Weeks"] = joined_train["Promo2Days"]//7
joined_train.loc[joined_train.Promo2Weeks<0, "Promo2Weeks"] = 0
joined_train.loc[joined_train.Promo2Weeks>25, "Promo2Weeks"] = 25
joined_train.Promo2Weeks.value_counts()

0     578880
25    403571
23      1659
21      1659
20      1659
19      1659
16      1659
15      1659
14      1659
24      1659
22      1659
18      1652
17      1633
13      1514
7       1267
6       1267
5       1263
8       1261
11      1260
10      1260
12      1260
9       1258
4       1236
3       1232
2       1232
1       1232
Name: Promo2Weeks, dtype: int64

In [59]:
joined_test["Promo2Since"] = pd.to_datetime(joined_test.apply(lambda x: Week(
    x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
joined_test["Promo2Days"] = joined_test.Date.subtract(joined_test["Promo2Since"]).dt.days
joined_test.loc[joined_test.Promo2Days<0, "Promo2Days"] = 0
joined_test.loc[joined_test.Promo2SinceYear<1990, "Promo2Days"] = 0
joined_test["Promo2Weeks"] = joined_test["Promo2Days"]//7
joined_test.loc[joined_test.Promo2Weeks<0, "Promo2Weeks"] = 0
joined_test.loc[joined_test.Promo2Weeks>25, "Promo2Weeks"] = 25
joined_test.Promo2Weeks.value_counts()

25    23472
0     17232
19       35
18       35
20       32
23       28
21       28
22       28
14       28
13       23
11       21
9        21
10       21
12       21
15       19
24       16
17       15
16        7
8         6
Name: Promo2Weeks, dtype: int64

In [62]:
joined_train.shape, joined_test.shape

((1017209, 76), (41088, 69))

In [63]:
# Remove dup columns
joined_train = joined_train.loc[:,~joined_train.columns.duplicated()]
joined_test = joined_test.loc[:,~joined_test.columns.duplicated()]

In [64]:
joined_train.shape, joined_test.shape

((1017209, 70), (41088, 67))

In [66]:
%time joined_train.to_feather('joined_train_before_durations.fth')

CPU times: user 1.29 s, sys: 3.49 s, total: 4.77 s
Wall time: 7.12 s


In [67]:
%time joined_test.to_feather('joined_test_before_durations.fth')

CPU times: user 84.2 ms, sys: 125 ms, total: 209 ms
Wall time: 199 ms


In [70]:
set(joined_train.columns) - set(joined_test.columns)

{'Customers', 'Sales', 'trend_y', 'week_y'}